In [14]:
import pandas as pd
import numpy as np
import math

### GET DATA

Dataverse will generate two table MC and FG as below:
- Model filter need_cut <-50  and find overlap Code of MC

In [3]:
# Get overlap params
fin_file_path = "../data/20240710_finish_df.xlsx"
fin_df = pd.read_excel(fin_file_path)
has_need_cut_df = fin_df[fin_df['need_cut'] < -10]
has_need_cut_df['params']= has_need_cut_df['maker'] + "+" + has_need_cut_df['spec_name']+ "+" + has_need_cut_df['thickness'].astype(str)
fin_params = has_need_cut_df['params'].unique()

mc_file_path = "../data/20240710_mc_df.xlsx"
mc_df = pd.read_excel(mc_file_path)
mc_df['params'] =  mc_df['maker'] + "+" + mc_df['spec_name']+ "+" + mc_df['thickness'].astype(str)
mc_params = mc_df['params'].unique()

# Find the intersection (overlapping values)
overlap = set(mc_params) & set(fin_params)

# Convert the result back to a list if needed
overlap_list = list(overlap)
overlap_list

C:\Users\thuduong\AppData\Local\Temp\ipykernel_2696\1691507548.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  has_need_cut_df['params']= has_need_cut_df['maker'] + "+" + has_need_cut_df['spec_name']+ "+" + has_need_cut_df['thickness'].astype(str)


['POSCO+JSH270C-PO+4.0',
 'CSVC+JSH270C-PO+2.0',
 'CSVC+JSH440W-PO+3.2',
 'POSCOVN+SPCC-SD+1.6',
 'POSCOVN+JSC270C-SD+1.6',
 'POSCO+SAPH440-PO+4.0',
 'POSCO+JSH270C-PO+3.2',
 'CSC+JSH590R-PO+2.3',
 'POSCO+JSH270C-PO+2.9',
 'CSC+JSH590R-PO+2.9',
 'POSCO+JSH440W-PO+2.0',
 'POSCOVN+JSC270C-SD+1.0',
 'POSCO+JSH270C-PO+2.6',
 'CSVC+JSH440W-PO+2.3',
 'CSC+JSH590R-PO+2.6',
 'POSCOVN+SPCC-SD+1.2',
 'POSCO+JSH270C-PO+2.3',
 'CSC+JSH590R-PO+2.0',
 'CSVC+JSH270C-PO+1.6',
 'CSC+JSC590R-SD+1.6',
 'POSCO+SS400-PO+2.3',
 'POSCOVN+SPCC-SD+1.0']

### PROCESS PARAMS

In [16]:
param = "POSCOVN+SPCC-SD+1.0"
param_split = param.split("+")

PARAMS = {"warehouse": "HSC"
          ,"spec_name": ""
          ,"thickness": 1
          ,"maker" : ""
          ,"stock_ratio": { 
                    "limited": None
                    # "default": 2
                    # "user_setting": 4
                }
        #   ,"forecast_scenario": median
          }

PARAMS["maker"] = param_split[0] 
PARAMS["spec_name"] = param_split[1]
PARAMS["thickness"] = float(param_split[2])

margin_dict = { 
    "HSC":
    {#save margin_dict in azure env, margin dictionary by warehouse
    "thickness_1.0": {
        "thickness": 1.0,
        "margin": 6
    },
    "thickness_2.9": {
        "thickness": 2.9,
        "margin": 6
    },
    "thickness_4": {
        "thickness": 4,
        "margin": 10
    },
    "thickness_2": {
        "thickness": 2,
        "margin": 6
    },
    "thickness_2.3": {
        "thickness": 2.3,
        "margin": 6
    },
    "thickness_1.6": {
        "thickness": 1.6,
        "margin": 6
    }
    },
    "NQS":
    {#save margin_dict in azure env, margin dictionary by warehouse
    "thickness_2.9": {
        "thickness": 2.9,
        "margin": 6
    },
    "thickness_2": {
        "thickness": 2,
        "margin": 6
    },
    "thickness_1.6": {
        "thickness": 1.6,
        "margin": 6
    }
    }
}

# GET ALL PARAMS
MIN_MARGIN = margin_dict[PARAMS["warehouse"]][f"thickness_{PARAMS["thickness"]}"]["margin"]
print(f"MIN_MARGIN:{MIN_MARGIN}")

BOUND_KEY = next(iter(PARAMS['stock_ratio']))
BOUND_VALUE = PARAMS['stock_ratio'][BOUND_KEY]
print(f"BOUND_VALUE:{BOUND_VALUE}")

MIN_MARGIN:6
BOUND_VALUE:None


#### Input & Process Data

In [17]:
PARAMS

{'warehouse': 'HSC',
 'spec_name': 'SPCC-SD',
 'thickness': 1.0,
 'maker': 'POSCOVN',
 'stock_ratio': {'limited': None}}

In [18]:
from process_df import filter_finish_df_to_dict, filter_by_params
from process_df import filter_multi_stock_df_to_dict, filter_by_params

file_path = "../data/20240710_finish_df.xlsx"
finish_key, value_columns = "order_id", ["width", "need_cut", f"upper_bound", "fc1", "fc2", "fc3"]
df = filter_by_params(file_path, PARAMS)
df = df[df['need_cut'] < -10]
finish = filter_finish_df_to_dict(df, finish_key, value_columns, BOUND_VALUE)

total_needcut = sum(item['need_cut'] for item in finish.values())
print(f'FG {len(finish)}, Need cut {total_needcut}')

finish

FG 7, Need cut 2195.876582278481


{'F200': {'width': 210.0,
  'need_cut': 908.876582278481,
  'upper_bound': 1585.750502278481,
  'fc1': 2256.2464,
  'fc2': 1984.6524,
  'fc3': 1934.0102},
 'F198': {'width': 152.0,
  'need_cut': 178.0,
  'upper_bound': 365.74,
  'fc1': 625.8,
  'fc2': 983.4,
  'fc3': 596.0},
 'F197': {'width': 150.0,
  'need_cut': 300.0,
  'upper_bound': 519.5526465,
  'fc1': 731.842155,
  'fc2': 687.165744375,
  'fc3': 577.24759125},
 'F196': {'width': 125.0,
  'need_cut': 100.0,
  'upper_bound': 141.505403125,
  'fc1': 138.35134375,
  'fc2': 130.7123125,
  'fc3': 48.38053125},
 'F190': {'width': 100.0,
  'need_cut': 300.0,
  'upper_bound': 445.829136,
  'fc1': 486.09712,
  'fc2': 812.15629,
  'fc3': 714.013235},
 'F511': {'width': 95.0,
  'need_cut': 300.0,
  'upper_bound': 358.212,
  'fc1': 194.04,
  'fc2': 183.48000000000002,
  'fc3': 329.99999999999994},
 'F203': {'width': 60.0,
  'need_cut': 109.0,
  'upper_bound': 520.96899538,
  'fc1': 1373.2299846,
  'fc2': 1331.7674778,
  'fc3': 905.3461206}}

In [19]:
file_path = "../data/20240710_mc_df.xlsx"
stock_key, value_columns = "inventory_id", ['receiving_date',"width", "weight"]
df = filter_by_params(file_path, PARAMS)
stocks = filter_multi_stock_df_to_dict(df, stock_key, value_columns)
print(len(stocks))
# list of stock >> list of need cut PO
# Improve: need rules to pick up stock to the pool

1


In [20]:
stocks

{'TP238H002948': {'receiving_date': 45261, 'width': 1219, 'weight': 6459}}

In [21]:
# save set STOCK- FINISHED GOOD to text
# Save the result to a text file
with open(f"{param}_data.txt", "w") as file:
    file.write(f"{param}\n")
    file.write(f"STOCK: \n {stocks}\n")
    file.write(f"FINISH: \n {finish}\n")